In [4]:
import numpy as np

class GMM:
  def __init__(self, n_components, max_iter=100, comp_names=None):
    self.n_components = n_components
    self.max_iter = max_iter

    if comp_names is None:
      self.comp_names = [f"comp_{i}" for i in range(n_components)]
    else:
      self.comp_names = comp_names

    self.pi = [1 / self.n_components for _ in range(self.n_components)]

  def multivariate_normal(self, X, mean_vector, covariance_matrix):
    d = len(mean_vector)
    det = np.linalg.det(covariance_matrix)
    if det == 0:
      det = 1e-6  # tránh chia 0

    norm_const = 1.0 / (np.power((2 * np.pi), d / 2) * np.sqrt(det))
    diff = X - mean_vector
    inv_cov = np.linalg.inv(covariance_matrix)
    exponent = -0.5 * np.dot(np.dot(diff.T, inv_cov), diff)

    return norm_const * np.exp(exponent)

  def fit(self, X):
    n_samples, n_features = X.shape

    # Khởi tạo ngẫu nhiên mean và covariance
    np.random.seed(42)
    self.means = [X[np.random.choice(n_samples)] for _ in range(self.n_components)]
    self.covs = [np.cov(X.T) for _ in range(self.n_components)]
    self.gamma = np.zeros((n_samples, self.n_components))

    for iteration in range(self.max_iter):
      # E-step: tính xác suất posterior
      for i in range(n_samples):
        probs = []
        for k in range(self.n_components):
          prob = self.pi[k] * self.multivariate_normal(X[i], self.means[k], self.covs[k])
          probs.append(prob)
        probs = np.array(probs)
        probs /= np.sum(probs)  # chuẩn hóa
        self.gamma[i] = probs

      # M-step: cập nhật tham số
      for k in range(self.n_components):
        N_k = np.sum(self.gamma[:, k])

        # Update mean
        self.means[k] = np.sum(self.gamma[:, k][:, np.newaxis] * X, axis=0) / N_k

        # Update covariance
        diff = X - self.means[k]
        self.covs[k] = np.dot((self.gamma[:, k][:, np.newaxis] * diff).T, diff) / N_k

        # Update pi
        self.pi[k] = N_k / n_samples

  def predict(self, X):
    n_samples = X.shape[0]
    y_pred = []

    for i in range(n_samples):
      probs = []
      for k in range(self.n_components):
        prob = self.pi[k] * self.multivariate_normal(X[i], self.means[k], self.covs[k])
        probs.append(prob)
      y_pred.append(np.argmax(probs))

    return np.array(y_pred)

In [10]:
# 1. Tạo dữ liệu giả lập
np.random.seed(42) # Đảm bảo kết quả lặp lại

# Cụm 1: Trung bình (0,0), hiệp phương sai gần hình cầu
mean1 = np.array([0., 0.])
cov1 = np.array([[1., 0.5], 
                 [0.5, 1.]])
X1 = np.random.multivariate_normal(mean1, cov1, 100) # 100 điểm

# Cụm 2: Trung bình (5,5), hiệp phương sai gần hình cầu
mean2 = np.array([5., 5.])
cov2 = np.array([[0.8, -0.2], 
                 [-0.2, 0.8]])
X2 = np.random.multivariate_normal(mean2, cov2, 100)

# Cụm 3: Trung bình (0,5), hiệp phương sai elip (dài theo trục y)
mean3 = np.array([0., 5.])
cov3 = np.array([[0.5, 0.], 
                 [0., 1.2]])
X3 = np.random.multivariate_normal(mean3, cov3, 100)

# Gom tất cả các điểm lại và trộn ngẫu nhiên
X_synthetic = np.vstack((X1, X2, X3))
np.random.shuffle(X_synthetic)

print("--- Bài tập 1: Giải với Class GMM của bạn ---")
print(f"Dữ liệu giả lập đã tạo có kích thước: {X_synthetic.shape}\n")

# 2. Khởi tạo và huấn luyện mô hình GMM
# Đặt số thành phần GMM là 3, tương ứng với 3 cụm dữ liệu đã tạo
gmm_model = GMM(n_components=3, max_iter=200) # Tăng max_iter để GMM có thời gian hội tụ
print("Bắt đầu huấn luyện GMM...")
gmm_model.fit(X_synthetic)
print("Huấn luyện GMM hoàn tất.\n")

--- Bài tập 1: Giải với Class GMM của bạn ---
Dữ liệu giả lập đã tạo có kích thước: (300, 2)

Bắt đầu huấn luyện GMM...
Huấn luyện GMM hoàn tất.



In [11]:
# 3. In ra các tham số đã học được của GMM
print("--- Các tham số của GMM sau huấn luyện ---")
print("Hệ số hỗn hợp (Weights / pi_k):\n", gmm_model.pi)

print("\nTrung bình (Means / mu_k):")
for i, mean in enumerate(gmm_model.means):
    print(f"Cụm {i}: {mean}")

print("\nMa trận hiệp phương sai (Covariances / Sigma_k):")
for i, cov in enumerate(gmm_model.covs):
    print(f"Cụm {i}:\n{cov}")

--- Các tham số của GMM sau huấn luyện ---
Hệ số hỗn hợp (Weights / pi_k):
 [0.09170939 0.60187069 0.30641992]

Trung bình (Means / mu_k):
Cụm 0: [1.15767474 5.38959946]
Cụm 1: [-0.01475829  2.25485933]
Cụm 2: [5.04215976 5.03451776]

Ma trận hiệp phương sai (Covariances / Sigma_k):
Cụm 0:
[[3.71305862 1.04600491]
 [1.04600491 0.45271241]]
Cụm 1:
[[ 0.63922241 -0.08526526]
 [-0.08526526  6.76255465]]
Cụm 2:
[[ 0.61843465 -0.26079398]
 [-0.26079398  0.67618584]]


In [12]:
# 4. Dự đoán nhãn cụm cho dữ liệu
print("\n--- Dự đoán và Phân tích Cụm ---")
predicted_labels = gmm_model.predict(X_synthetic)
print(f"Ví dụ 20 nhãn dự đoán đầu tiên: {predicted_labels[:20]}")



--- Dự đoán và Phân tích Cụm ---
Ví dụ 20 nhãn dự đoán đầu tiên: [1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2]


In [13]:
# 5. In ra số lượng điểm trong mỗi cụm
# np.bincount đếm số lần xuất hiện của mỗi giá trị không âm trong một mảng.
unique_labels, counts = np.unique(predicted_labels, return_counts=True)
print("\nSố lượng điểm trong mỗi cụm:")
for label, count in zip(unique_labels, counts):
    print(f"Cụm {label}: {count} điểm")



Số lượng điểm trong mỗi cụm:
Cụm 0: 15 điểm
Cụm 1: 191 điểm
Cụm 2: 94 điểm
